##  Ejercicio 2
### Actividad 1
Escribir un módulo en Verilog que permita realizar una suma de dos entradas en punto
fijo con los siguientes formatos: S16.14 y S12.11, entregando diferentes salidas en los
siguientes formatos.
* Full-resolution
* S11.10 con overflow y truncado.
* S11.10 con saturación y truncado.
* S9.8 con saturación y redondeo.

#### Descripción en verilog

---
```verilog
module sumador_fixed 
#(
    parameter NBA  =16, parameter  NBFA  =14, // Entrada A S(xx,xx)
    parameter NBB  =12, parameter  NBFB  =11, // Entrada B S(xx,xx)
    parameter NBS1 =11,  parameter NBFS1 =10, // Salida  1 S(xx,xx)
    parameter NBS2 =9 ,  parameter NBFS2 =8   // Salida  2 S(xx,xx)
)
(
    input signed   [NBA-1:0]  i_saa_aa,              //S(16,14) S,E,14D
    input signed   [NBB-1:0]  i_sbb_bb,              //S(12,11) S,11D
    output signed  [NBA:0]    o_sxx_xx_full,         //S(17,14)C,S,E,14D
    output signed  [NBS1-1:0] o_s11_11_over_trunc,   //S(11,10)
    output signed  [NBS1-1:0] o_s11_11_satu_trunc,   //S(11,10)
    output signed  [NBS2-1:0] o_s22_22_satu_round    //S(9,8)
);
localparam NEA   =  NBA  - NBFA;
localparam NEB   =  NBB  - NBFB;
localparam NES1  =  NBS1 - NBFS1;
localparam NES2  =  NBS2 - NBFS2;

localparam DOT   =  NEA-NBFA-1;

localparam DEAB  =  NEA  - NEB;
localparam DFAB  =  NBFA - NBFB;


wire signed [NBA  :0] s_full;
wire signed [NBA-1:0] s_aj_sbb_bb;
wire signed [NBS1 :0] s_trunc_sat;

wire signed [NBS2  :0] s_s22_22;
wire signed [NBS2+1:0] s_s22_22_round;
wire signed [NBS2-1:0]  s_s22_22_round_satu;

assign s_aj_sbb_bb = {{DEAB{i_sbb_bb[NBB-1]}},i_sbb_bb,{DFAB{1'b0}}}; 

assign s_full = i_saa_aa + s_aj_sbb_bb; // Alternativa reemplazar s_aj_sbb_bb por $signed({i_sbb_bb[11],i_sbb_bb,3'b000})

assign s_trunc_sat = &s_full[NBA : NEA-DOT-NES1] || ~|s_full[NBA : NEA-DOT-NES1] ? s_full[NEA-DOT-NES1 -: NBS1]:
                     &s_full[NBA]                                                ? {1'b1,{10{1'b0}}} : 
                                                                                   {1'b0,{10{1'b1}}} ; 

assign s_s22_22 = s_full[NEA-DOT-NES2 -:NBS2+1];
assign s_s22_22_round = s_s22_22 + 1'b1;

assign s_s22_22_round_satu = &s_s22_22_round[NBS2+1:NBS2] || ~|s_s22_22_round[NBS2+1:NBS2] ? s_s22_22_round[NBS2:1] :
                             &s_s22_22_round[NBS2+1]                            ? {1'b1,{NBFS2{1'b0}}} : 
                                                                              {1'b0,{NBFS2{1'b1}}} ; 

assign o_sxx_xx_full = s_full;
assign o_s11_11_over_trunc = s_full[NEA-DOT-NES1 -: NBS1];
assign o_s11_11_satu_trunc = s_trunc_sat;
assign o_s22_22_satu_round = s_s22_22_round_satu;

endmodule

```
---

#### Testbench

#### Generador de vectores.

In [2]:
import math
import numpy as np
from tool._fixedInt import *

In [3]:
i_a_rand = np.random.uniform(low=-0.9, high=0.9, size=(5))
i_b_rand = np.random.uniform(low=-0.9, high=0.9, size=(5))
i_saa_aa = arrayFixedInt(16,14, i_a_rand, signedMode='S', roundMode='round', saturateMode='wrap')
i_sbb_bb = arrayFixedInt(12,11, i_b_rand, signedMode='S', roundMode='round', saturateMode='wrap')
o_sxx_xx_full_sum=DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=17,fractWidth=14,saturateMode='saturate') # Definicion completa 
o_s11_11_over_trunc_c=DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=11,fractWidth=10,saturateMode='wrap') # Definicion completa 
o_s11_11_satu_trunc_c=DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=11,fractWidth=10,saturateMode='saturate') # Definicion completa 
o_s22_22_satu_round_c=DeFixedInt(roundMode='round',signedMode = 'S',totalWidth=9,fractWidth=8,saturateMode='saturate') # Definicion completa 
o_saa_aa=""
o_sbb_bb=""
o_sxx_xx_full=""
o_s11_11_over_trunc=""
o_s11_11_satu_trunc=""
o_s22_22_satu_round=""
for i in range(len(i_saa_aa)):
    o_saa_aa+=str((bin(i_saa_aa[i].intvalue))[2:].zfill(i_saa_aa[i].width))+'\n'
    o_sbb_bb+=str((bin(i_sbb_bb[i].intvalue))[2:].zfill(i_sbb_bb[i].width))+'\n'
    o_sxx_xx_full_sum.assign(i_saa_aa[i]+i_sbb_bb[i])
    o_s11_11_over_trunc_c.value=o_sxx_xx_full_sum.fValue
    o_s11_11_satu_trunc_c.value=o_sxx_xx_full_sum.fValue
    o_s22_22_satu_round_c.value=o_sxx_xx_full_sum.fValue
    o_sxx_xx_full+=str((bin(o_sxx_xx_full_sum.intvalue))[2:].zfill(o_sxx_xx_full_sum.width))+'\n'
    o_s11_11_over_trunc+=str((bin(o_s11_11_over_trunc_c.intvalue))[2:].zfill(o_s11_11_over_trunc_c.width))+'\n'
    o_s11_11_satu_trunc+=str((bin(o_s11_11_satu_trunc_c.intvalue))[2:].zfill(o_s11_11_satu_trunc_c.width))+'\n'
    o_s22_22_satu_round+=str((bin(o_s22_22_satu_round_c.intvalue))[2:].zfill(o_s22_22_satu_round_c.width))+'\n'

f = open ('test_files/py_i_saa_aa.txt','w')
f.write(o_saa_aa)
f.close()
f = open ('test_files/py_i_sbb_bb.txt','w')
f.write(o_sbb_bb)
f.close()
f = open ('test_files/py_o_sxx_xx_full.txt','w')
f.write(o_sxx_xx_full)
f.close()
f = open ('test_files/py_o_s11_11_over_trunc.txt','w')
f.write(o_s11_11_over_trunc)
f.close()
f = open ('test_files/py_o_s11_11_satu_trunc.txt','w')
f.write(o_s11_11_satu_trunc)
f.close()
f = open ('test_files/py_o_s22_22_satu_round.txt','w')
f.write(o_s22_22_satu_round)
f.close()
print("Vectores generados.")

Vectores generados.


#### comparador de vectores

In [4]:
path='test_files/'
py_vectors=['py_o_sxx_xx_full.txt','py_o_s11_11_over_trunc.txt','py_o_s11_11_satu_trunc.txt','py_o_s22_22_satu_round.txt']
v_vectors  = ["v_o_sxx_xx_full.txt",'v_o_s11_11_over_trunc.txt','v_o_s11_11_satu_trunc.txt','v_o_s22_22_satu_round.txt']
for i in range(len(py_vectors)):
    archivo = open(path+py_vectors[i], 'r')
    py_vector_temp = archivo.read().split('\n')
    archivo.close()
    archivo = open(path+v_vectors[i], 'r')
    V_vector_temp = archivo.read().split('\n')
    archivo.close()
    print('---- matching '+py_vectors[i]+' with '+v_vectors[i]+'----')
    flag=0
    for j in range(len(py_vector_temp)):
        if(py_vector_temp[j]== V_vector_temp[j]):
            pass
        else:
            print('index '+str(j)+' python: '+ py_vector_temp[j]+ ' verilog: ' + V_vector_temp[j])
            flag=1
    if not flag:
        print("all Passed.")


---- matching py_o_sxx_xx_full.txt with v_o_sxx_xx_full.txt----
index 0 python: 00011010111110001 verilog: 11110011000111111
index 1 python: 00011001000111010 verilog: 11011110001101000
index 2 python: 00001001001000111 verilog: 11111111001110011
index 3 python: 11110110111111000 verilog: 00001100000001011
index 4 python: 11111111110011100 verilog: 00010111101101010
---- matching py_o_s11_11_over_trunc.txt with v_o_s11_11_over_trunc.txt----
index 0 python: 01101011111 verilog: 11001100011
index 1 python: 01100100011 verilog: 01111000110
index 2 python: 00100100100 verilog: 11111100111
index 3 python: 11011011111 verilog: 00110000000
index 4 python: 11111111001 verilog: 01011110110
---- matching py_o_s11_11_satu_trunc.txt with v_o_s11_11_satu_trunc.txt----
index 0 python: 01101011111 verilog: 11001100011
index 1 python: 01100100011 verilog: 10000000000
index 2 python: 00100100100 verilog: 11111100111
index 3 python: 11011011111 verilog: 00110000000
index 4 python: 11111111001 verilog: 0

### Actividad 2
Escribir un módulo en Verilog que permita realizar una multiplicación de dos entradas
en punto fijo con los siguientes formatos: S8.6 y S12.11, entregando diferentes salidas
en los siguientes formatos.
* Full-resolution.
* S12.11 con overflow y truncado.
* S12.11 con saturación y truncado.
* S10.9 con saturación y redondeo.


#### Descripción en verilog.

---
```verilog
module multiplicador_fixed 
#(
    parameter NBA  =12, parameter  NBFA  =11, // Entrada A S(xx,xx)
    parameter NBB  =08, parameter  NBFB  =06,  // Entrada B S(xx,xx)
    parameter NBS1 =12, parameter  NBFS1 =11, // Salida  1 S(xx,xx)
    parameter NBS2 =10 , parameter NBFS2 =09   // Salida  2 S(xx,xx)
)
(
    input signed   [NBA-1:0]        i_saa_aa,              
    input signed   [NBB-1:0]        i_sbb_bb,              
    output signed  [NBA+NBB-1 :0]   o_sxx_xx_full,        
    output signed  [NBS1-1:0]       o_s11_11_over_trunc,   
    output signed  [NBS1-1:0]       o_s11_11_satu_trunc,   
    output signed  [NBS2-1:0]       o_s22_22_satu_round    
);
localparam NEA   =  NBA  - NBFA;
localparam NEB   =  NBB  - NBFB;
localparam NES1  =  NBS1 - NBFS1;
localparam NES2  =  NBS2 - NBFS2;
localparam NEM   =  NBA+NBB -1;
localparam DOT   =  NEM-NBFA-NBFB-1;

wire signed [NEM:0]    s_full;
wire signed [NBS1 :0]  s_trunc_sat;

wire signed [NBS2  :0] s_s22_22;
wire signed [NBS2+1:0] s_s22_22_round;
wire signed [NBS2-1:0] s_s22_22_round_satu;



assign s_full = i_saa_aa * i_sbb_bb;

assign s_trunc_sat = &s_full[NEM : NEM-DOT-NES1] || ~|s_full[NEM : NEM-DOT-NES1] ? s_full[NEM-DOT-NES1 -: NBS1]:
                     &s_full[NEM]                                                ? {1'b1,{10{1'b0}}} : 
                                                                                   {1'b0,{10{1'b1}}} ; 

assign s_s22_22 = s_full[NEM-DOT-NES2 -:NBS2+1];
assign s_s22_22_round = s_s22_22 + 1'b1;

assign s_s22_22_round_satu = &s_s22_22_round[NBS2+1:NBS2] || ~|s_s22_22_round[NBS2+1:NBS2] ? s_s22_22_round[NBS2:1] :
                             &s_s22_22_round[NBS2+1]                            ? {1'b1,{NBFS2{1'b0}}} : 
                                                                                  {1'b0,{NBFS2{1'b1}}} ; 

assign o_sxx_xx_full = s_full;
assign o_s11_11_over_trunc = s_full[NEM-DOT-NES1 -: NBS1];
assign o_s11_11_satu_trunc = s_trunc_sat;
assign o_s22_22_satu_round = s_s22_22_round_satu;

endmodule
```
---

#### Testbench

---
```verilog
`timescale 1 ns/10 ps  // time-unit = 1 ns, precision = 10 ps
`define PATH "C:/Users/matia/Documents/UTN/0_Extracurricular/0_Fundacion_Fulgor/Diseno_Digital/tp4/entregable/test_files/"
module tb_multiplicador_fixed;
    parameter NBA  =12;  parameter  NBFA  =11;   // Entrada A S(xx,xx)
    parameter NBB  =08;  parameter  NBFB  =06;   // Entrada B S(xx,xx)
    parameter NBS1 =12;  parameter  NBFS1 =11;   // Salida  1 S(xx,xx)
    parameter NBS2 =10;  parameter  NBFS2 =09;   // Salida  2 S(xx,xx)
    parameter path_dir = `PATH;
    reg signed   [NBA-1:0]  i_saa_aa; 
    reg signed   [NBB-1:0]  i_sbb_bb; 
    reg          [NBA+NBB-1 :0]    o_sxx_xx_full_w; 
    reg          [NBS1-1:0]        o_s11_11_over_trunc_w;          
    reg          [NBS1-1:0]        o_s11_11_satu_trunc_w;          
    reg          [NBS2-1:0]        o_s22_22_satu_round_w;          
    wire         [NBA+NBB-1 :0]    o_sxx_xx_full;          
    wire         [NBS1-1:0]        o_s11_11_over_trunc;    
    wire         [NBS1-1:0]        o_s11_11_satu_trunc;    
    wire         [NBS2-1:0]        o_s22_22_satu_round;    
    reg signed   [NBA-1:0]         read_data_i_saa_aa [0:5];
    reg signed   [NBB-1:0]         read_data_i_sbb_bb [0:5];
    integer tb_o_sxx_xx_full_dir;
    integer tb_o_s11_11_over_trunc_dir;
    integer tb_o_s11_11_satu_trunc_dir;
    integer tb_s22_22_satu_round_dir;
    integer i;
    initial
    begin 
        $readmemb({path_dir,"py_i_saa_aa.txt"}, read_data_i_saa_aa);
        $readmemb({path_dir,"py_i_sbb_bb.txt"}, read_data_i_sbb_bb);
        tb_o_sxx_xx_full_dir       = $fopen({path_dir,"v_o_sxx_xx_full.txt"});
        tb_o_s11_11_over_trunc_dir = $fopen({path_dir,"v_o_s11_11_over_trunc.txt"});
        tb_o_s11_11_satu_trunc_dir = $fopen({path_dir,"v_o_s11_11_satu_trunc.txt"});
        tb_s22_22_satu_round_dir   = $fopen({path_dir,"v_o_s22_22_satu_round.txt"});

        for (i=0; i<5; i=i+1)
        begin
            i_saa_aa = read_data_i_saa_aa[i]; 
            i_sbb_bb = read_data_i_sbb_bb[i]; 
            o_sxx_xx_full_w <= o_sxx_xx_full;
            o_s11_11_over_trunc_w<=  o_s11_11_over_trunc;      
            o_s11_11_satu_trunc_w<=  o_s11_11_satu_trunc;    
            o_s22_22_satu_round_w<=  o_s22_22_satu_round;       
            #2;  
            o_sxx_xx_full_w <= o_sxx_xx_full;
            o_s11_11_over_trunc_w<=  o_s11_11_over_trunc;      
            o_s11_11_satu_trunc_w<=  o_s11_11_satu_trunc;    
            o_s22_22_satu_round_w<=  o_s22_22_satu_round;  
            #2;  
            $fdisplay(tb_o_sxx_xx_full_dir, "%b",o_sxx_xx_full_w);
            $fdisplay(tb_o_s11_11_over_trunc_dir, "%b",o_s11_11_over_trunc_w);
            $fdisplay(tb_o_s11_11_satu_trunc_dir, "%b",o_s11_11_satu_trunc_w);
            $fdisplay(tb_s22_22_satu_round_dir, "%b",o_s22_22_satu_round_w);
            #100;  
        end
        $fclose(tb_o_sxx_xx_full_dir);
        $fclose(tb_o_s11_11_over_trunc_dir);
        $fclose(tb_o_s11_11_satu_trunc_dir);
        $fclose(tb_s22_22_satu_round_dir);
    end

multiplicador_fixed
    u_multiplicador_fixed(
        .i_saa_aa(i_saa_aa), 
        .i_sbb_bb(i_sbb_bb),  
        .o_sxx_xx_full(o_sxx_xx_full),         
        .o_s11_11_over_trunc(o_s11_11_over_trunc),    
        .o_s11_11_satu_trunc(o_s11_11_satu_trunc),    
        .o_s22_22_satu_round(o_s22_22_satu_round)    
    );
endmodule
```
---

#### Generador de vectores.

In [5]:
i_a_rand = np.random.uniform(low=-0.9, high=0.9, size=(5))
i_b_rand = np.random.uniform(low=-0.9, high=0.9, size=(5))
i_saa_aa = arrayFixedInt(12,11, i_a_rand, signedMode='S', roundMode='round', saturateMode='wrap')
i_sbb_bb = arrayFixedInt(8,6, i_b_rand, signedMode='S', roundMode='round', saturateMode='wrap')
o_sxx_xx_full_op = DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=20,fractWidth=17,saturateMode='saturate') # Definicion completa 
o_s11_11_over_trunc_c=DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=12,fractWidth=11,saturateMode='wrap') # Definicion completa 
o_s11_11_satu_trunc_c=DeFixedInt(roundMode='trunc',signedMode = 'S',totalWidth=12,fractWidth=11,saturateMode='saturate') # Definicion completa 
o_s22_22_satu_round_c=DeFixedInt(roundMode='round',signedMode = 'S',totalWidth=10,fractWidth=9,saturateMode='saturate') # Definicion completa 
o_saa_aa=""
o_sbb_bb=""
o_sxx_xx_full=""
o_s11_11_over_trunc=""
o_s11_11_satu_trunc=""
o_s22_22_satu_round=""
for i in range(len(i_saa_aa)):
    o_saa_aa+=str((bin(i_saa_aa[i].intvalue))[2:].zfill(i_saa_aa[i].width))+'\n'
    o_sbb_bb+=str((bin(i_sbb_bb[i].intvalue))[2:].zfill(i_sbb_bb[i].width))+'\n'
    o_sxx_xx_full_op.assign(i_saa_aa[i]*i_sbb_bb[i])
    o_s11_11_over_trunc_c.value=o_sxx_xx_full_op.fValue
    o_s11_11_satu_trunc_c.value=o_sxx_xx_full_op.fValue
    o_s22_22_satu_round_c.value=o_sxx_xx_full_op.fValue
    o_sxx_xx_full+=str((bin(o_sxx_xx_full_op.intvalue))[2:].zfill(o_sxx_xx_full_op.width))+'\n'
    o_s11_11_over_trunc+=str((bin(o_s11_11_over_trunc_c.intvalue))[2:].zfill(o_s11_11_over_trunc_c.width))+'\n'
    o_s11_11_satu_trunc+=str((bin(o_s11_11_satu_trunc_c.intvalue))[2:].zfill(o_s11_11_satu_trunc_c.width))+'\n'
    o_s22_22_satu_round+=str((bin(o_s22_22_satu_round_c.intvalue))[2:].zfill(o_s22_22_satu_round_c.width))+'\n'

f = open ('test_files/py_i_saa_aa.txt','w')
f.write(o_saa_aa)
f.close()
f = open ('test_files/py_i_sbb_bb.txt','w')
f.write(o_sbb_bb)
f.close()
f = open ('test_files/py_o_sxx_xx_full.txt','w')
f.write(o_sxx_xx_full)
f.close()
f = open ('test_files/py_o_s11_11_over_trunc.txt','w')
f.write(o_s11_11_over_trunc)
f.close()
f = open ('test_files/py_o_s11_11_satu_trunc.txt','w')
f.write(o_s11_11_satu_trunc)
f.close()
f = open ('test_files/py_o_s22_22_satu_round.txt','w')
f.write(o_s22_22_satu_round)
f.close()
print("Vectores generados.")

Vectores generados.


#### Comparador de vectores

In [6]:
path='test_files/'
py_vectors=['py_o_sxx_xx_full.txt','py_o_s11_11_over_trunc.txt','py_o_s11_11_satu_trunc.txt','py_o_s22_22_satu_round.txt']
v_vectors  = ["v_o_sxx_xx_full.txt",'v_o_s11_11_over_trunc.txt','v_o_s11_11_satu_trunc.txt','v_o_s22_22_satu_round.txt']
for i in range(len(py_vectors)):
    archivo = open(path+py_vectors[i], 'r')
    py_vector_temp = archivo.read().split('\n')
    archivo.close()
    archivo = open(path+v_vectors[i], 'r')
    V_vector_temp = archivo.read().split('\n')
    archivo.close()
    print('---- matching '+py_vectors[i]+' with '+v_vectors[i]+'----')
    flag=0
    for j in range(len(py_vector_temp)):
        if(py_vector_temp[j]== V_vector_temp[j]):
            pass
        else:
            print('index '+str(j)+' python: '+ py_vector_temp[j]+ ' verilog: ' + V_vector_temp[j])
            flag=1
    if not flag:
        print("all Passed.")


---- matching py_o_sxx_xx_full.txt with v_o_sxx_xx_full.txt----
index 0 python: 11111011100011001100 verilog: 11110011000111111
index 1 python: 00000001110111101010 verilog: 11011110001101000
index 2 python: 00000011111111010101 verilog: 11111111001110011
index 3 python: 00000010010011011110 verilog: 00001100000001011
index 4 python: 00001100110100111110 verilog: 00010111101101010
---- matching py_o_s11_11_over_trunc.txt with v_o_s11_11_over_trunc.txt----
index 0 python: 111011100011 verilog: 11001100011
index 1 python: 000001110111 verilog: 01111000110
index 2 python: 000011111111 verilog: 11111100111
index 3 python: 000010010011 verilog: 00110000000
index 4 python: 001100110100 verilog: 01011110110
---- matching py_o_s11_11_satu_trunc.txt with v_o_s11_11_satu_trunc.txt----
index 0 python: 111011100011 verilog: 11001100011
index 1 python: 000001110111 verilog: 10000000000
index 2 python: 000011111111 verilog: 11111100111
index 3 python: 000010010011 verilog: 00110000000
index 4 python